In [1]:
import pandas as pd
import os
import sys
from tqdm import tqdm
import json
import shutil

json_files = []
searchDir = './metaData/unprocessed'

for item in os.listdir(searchDir):
    json_files.append(item)
print(json_files)

['guys.json', 'sexist.json', 'everydaysexism.json', 'metoo.json', 'bloodymen.json', 'girls.json', 'boys.json', 'manspalining.json', 'slutshaming.json', 'sexism.json']


In [6]:
texts = list()
with open(os.path.join(searchDir, 'bloodymen.json'), 'r') as f:
        distros_dict = json.load(f)
        for i in range(len(distros_dict['GraphImages'])):
            try:
                texts.append(distros_dict['GraphImages'][i]['edge_media_to_caption']['edges'][0]['node']['text'])
            except IndexError:
                continue
texts

['The debate between what is considered a stereotype and what is prejudice is involved and ongoing. Why is it acceptable to stereotype women, POC, LGBTQIA etc but the second you generalise men being shit we hear NOT ALL DAVES!!! IVE NEVER RAPED A WOMAN!!! IM A NICE GUY!! WOMEN DO THAT TOO!! Hmmm it’s like the only acceptable generalisations are for non-white, non-straight, non-men. 🤔🤔🤔 Like I’m so bored of this ignorance. I’m constantly receiving abuse, harassment and being mansplained to because of this account and it only spurs me forthwith into fighting you rape-culture-facilitators more! They call me a cunt and tell me to suck their dick whilst they tell me I’m angry and damaging their world. No Dave, you just know every woman I bring over to the dark side means one less woman willing to put up with your shit! #notalldaves #notallmen #trolls #bloodymen #fuckthepatriarchy #destroythepatriarchy #feminism #mansplaining #feminist #rude #jerks #maleprivilege #abuse #thesydneyfeminists',

In [8]:
import preprocessor as p
from langdetect import detect
cleaned_texts = list()
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.HASHTAG, p.OPT.MENTION)

for tx in texts:
    tx = p.clean(tx)
    try:
        if len(tx.split()) > 5 and detect(tx) == 'en':
            cleaned_texts.append(tx)
    except:
        continue

len(set(cleaned_texts))

312

In [9]:
all_texts = list()
for file in json_files:
    with open(os.path.join(searchDir, file), 'r') as f:
        distros_dict = json.load(f)
        i = 0
        for i in range(len(distros_dict['GraphImages'])):
            try:
                caption = p.clean(distros_dict['GraphImages'][i]['edge_media_to_caption']['edges'][0]['node']['text'])
                if len(caption.split()) > 5 and detect(caption) == 'en':
                    all_texts.append(caption)
            except:
                continue
                
all_texts = list(set(all_texts))
all_texts

["Still cant get over how beautiful and valid this is. with ・・・ “...and I do think that there are other ways that we could live and be. And beyond that, I think we deserve it, we have to demand it, we have to believe that it’s possible. We have to believe in that world, it can’t be a utopia.” ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ I've put the entirety of my conversation with on my IGTV ️.",
 'Louder for the people in the back',
 'Check him sis! The Tea is piping hot y’all! \U0001f975 ️ tune in to epi 2.6 where the Qwns chat about love, sex, and dating — here, spills the tea on an ex that was slut shaming her for sleeping with HIM on the 1st date. Um??? 🤔🤔 thoughts from our thots?? Click the link in bio to listen! PS: who can count how many times Jerralyn says “like” lol PPS: NAME THAT BRITNEY SONG!',
 'Reposted from () - Yes! -',
 'What Ho Chaps and Chappesses! How are you? Looking fabulous today! Ding Dong! Let\'s talk about Catherine of Aragon. Poor old cow had a rotten life. She married a chap 

In [10]:
import emoji

x = []
for w in all_texts[1].split():
    w = emoji.demojize(w)
    if ':' not in w:
        x.append(w)
s = ' '.join(x)
s

'Louder for the people in the back'

In [12]:
cleaned_texts = []
score = []
for t in all_texts:
    x = []
    for w in t.split():
        w = emoji.demojize(emoji.emojize(w))
        if ':' not in w and '_' not in w:
            w = "".join(filter(lambda x: ord(x)<128, w))
            x.append(w)
    cleaned_texts.append(' '.join(x))
    score.append(0)
        
cleaned_texts

["Still cant get over how beautiful and valid this is. with  ...and I do think that there are other ways that we could live and be. And beyond that, I think we deserve it, we have to demand it, we have to believe that its possible. We have to believe in that world, it cant be a utopia.   I've put the entirety of my conversation with on my IGTV .",
 'Louder for the people in the back',
 'Check him sis! The Tea is piping hot yall!   tune in to epi 2.6 where the Qwns chat about love, sex, and dating  here, spills the tea on an ex that was slut shaming her for sleeping with HIM on the 1st date. Um??? thoughts from our thots?? Click the link in bio to listen! who can count how many times Jerralyn says like lol NAME THAT BRITNEY SONG!',
 'Reposted from () - Yes! -',
 'What Ho Chaps and Chappesses! How are you? Looking fabulous today! Ding Dong! Let\'s talk about Catherine of Aragon. Poor old cow had a rotten life. She married a chap called Prince Arthur who died a year into their marriage. B

In [14]:
import pandas as pd
c_dict = {'texts': cleaned_texts, 'scores': score}
pd.DataFrame(c_dict).to_csv('larger_texts.csv', index=False)